In [1]:
import math
import metapy
import sys
import time
import pytoml

In [2]:
idx = metapy.index.make_inverted_index('config.toml')
idx.num_docs()
idx.unique_terms()

579

In [3]:
fidx = metapy.index.make_forward_index('config.toml')
fidx.num_labels()

2

In [4]:
dset = metapy.classify.MulticlassDataset(fidx)
len(dset)

19

In [5]:
set([dset.label(instance) for instance in dset])

{'investment', 'movein'}

In [6]:
view = dset[0:len(dset)+1]
view = metapy.classify.MulticlassDatasetView(dset)
view.shuffle()
print("{} vs {}".format(view[0].id, dset[0].id))

16 vs 0


In [7]:
training = view[0:int(0.75*len(view))]
testing = view[int(0.75*len(view)):len(view)+1]

In [8]:
nb = metapy.classify.NaiveBayes(training)

In [9]:
mtrx = nb.test(testing)
print(mtrx)


              investment  movein      
            ------------------------
 investment | 1           -           
     movein | 0.5         0.5         




In [10]:
mtrx.print_stats()

------------------------------------------------------------
Class       F1 Score    Precision   Recall      Class Dist  
------------------------------------------------------------
investment  0.857       0.75        1           0.6         
movein      0.667       1           0.5         0.4         
------------------------------------------------------------
Total       0.824       0.85        0.8         
------------------------------------------------------------
5 predictions attempted, overall accuracy: 0.8



In [11]:
idx.num_docs()
idx.avg_doc_length()

89.10526275634766

In [12]:
idx.total_corpus_terms()


1693

In [15]:
query = metapy.index.Document()
ranker = metapy.index.OkapiBM25(k1=1.2,b=0.75,k3=500)
num_results = 10
ev = metapy.index.IREval('config.toml')
with open('config.toml', 'r') as fin:
        cfg_d = pytoml.load(fin)
        
query_cfg = cfg_d['query-runner']
query_start = query_cfg.get('query-id-start', 0)
    
with open('investment-queries.txt') as query_file:
    for query_num, line in enumerate(query_file):
        query.content(line.strip())
        print(query.content())
        results = ranker.score(idx, query, num_results)
        print(results)
        print(num_results)
        avg_p = ev.avg_p(results, query_start + query_num, num_results)
        print("Query {} average precision: {}".format(query_num + 1, avg_p))
ev.map()

potential scope refurbishment value stamp modernisation improvement need convert future
[(6, 5.730287551879883), (3, 5.7234954833984375), (0, 3.951308250427246), (4, 3.904705762863159), (5, 3.5449962615966797), (1, 2.516573190689087), (9, 2.180393695831299), (2, 2.0118000507354736), (7, 1.9803178310394287), (15, 1.8454886674880981)]
10
Query 1 average precision: 0.7004365079365078


0.7004365079365078